The RBF network only has one layer, and it implements a weighted combination of radial basis functions whose derivatives can be easily computed analytically. This means that the gradient of the RBF network and its curvature (Hessian) can be computed analytically and there is no need for costly automatic differentiation.

In [10]:

from sympy import *

# Evaluation point of the Thin Plate Spline
p1, p2, p3 = symbols("p1 p2 p3")

# A center of the Thin Plate Spline
c1, c2, c3 = symbols("c1 c2 c3")

# Radius (Eucledian distance) between the evaluation (data) 
# point and the center
r = sqrt((p1 - c1)**2 + (p2 - c2)**2 + (p3 - c3)**2)

# A real-number that stabilizes the TPS RBF if the data point
# and the centroid overlap.  
epsilon = symbols("epsilon")

# Thin Plate Spline Radial Basis Function - TPS RBF
phi = r**2*ln(r + epsilon)
phi

((-c1 + p1)**2 + (-c2 + p2)**2 + (-c3 + p3)**2)*log(epsilon + sqrt((-c1 + p1)**2 + (-c2 + p2)**2 + (-c3 + p3)**2))

RBF function gradient

These expressions we need in the loss term when approximating the components of the displacement vector field. The idea is to compute the [strain rate tensor](https://en.wikipedia.org/wiki/Strain-rate_tensor) from the displacement field, and constrain it. Minimizing shear in the displacement Jacobian ("strain rate tensor") should, hypothetically, result in a displacement field that causes less mesh non-orthogonality.  

Rate of shear: 

$$\tau_{ij} = \frac{1}{2}(\partial_i d_j + \partial_j d_i)$$


How to minimize mesh skewness is still open.

In [19]:
# Substitute the long expression for the radius with R 
R = symbols("R")
partial_p1_phi = diff(phi, p1)
partial_p1_phi_nice = partial_p1_phi.subs(sqrt((p1 - c1)**2 + (p2 - c2)**2 + (p3 - c3)**2), R).simplify()
partial_p1_phi_nice

(-R - 2*(R + epsilon)*log(R + epsilon))*(c1 - p1)/(R + epsilon)

In [17]:
partial_p2_phi = diff(phi, p2)
partial_p2_phi_nice = partial_p2_phi.subs(sqrt((p1 - c1)**2 + (p2 - c2)**2 + (p3 - c3)**2), R).simplify()
partial_p2_phi_nice

(-R - 2*(R + epsilon)*log(R + epsilon))*(c2 - p2)/(R + epsilon)

In [20]:
partial_p3_phi = diff(phi, p3)
partial_p3_phi_nice = partial_p3_phi.subs(sqrt((p1 - c1)**2 + (p2 - c2)**2 + (p3 - c3)**2), R).simplify()
partial_p3_phi_nice

(-R - 2*(R + epsilon)*log(R + epsilon))*(c3 - p3)/(R + epsilon)

The gradient components are simple terms of radius R and the components of the point and each RBF center. The pattern is clear if we increase/decrease vector dimensions, so broadcasting can be used to compute the components. 

**Note**: RBF approximation is a scalar approximation, that must be done for each component of the displacement vector field separately. 

This means, if $\phi$ approximates the first component $\delta_1$ of our displacement field $\mathbf{\delta} = [\delta_1, \delta_2, \delta_3]$, we need only to compute $\partial_{p_2} \phi$ and $\partial_{p_3} \phi$ and to minimize $0.5|\partial_{p_2} \delta_1 + \partial_{p_3} \delta_1|$. In the case of the first component of the displacement $\partial_{p_1} \phi = \partial_{p_1} \delta_1$ is a component responsible for compression (divergence), we don't need this to minimize non-orthogonality. The approach is equivalent for $\delta_2$ and $\delta_3$. 

**Note**: up to this point we are only looking into a single RBF, the RBF network is a weighted sum of RBFs!

Hessian of the RBF

If we don't apply smoothness constraints on Thin Plate Spline RBF approximation, it will overfit data - the idea of the spline was originally to [apply constraints on it](https://en.wikipedia.org/wiki/Flat_spline) when engineering shapes. Thin-plate spline constraint is its curvature, [_its smoothnes measure_](https://en.wikipedia.org/wiki/Thin_plate_spline). For this we need second derivatives of $\phi$. 

In [23]:
partial2_p1_phi = diff(partial_p1_phi, p1)
partial2_p1_phi_nice = partial2_p1_phi.subs(sqrt((p1 - c1)**2 + (p2 - c2)**2 + (p3 - c3)**2), R).simplify()
partial2_p1_phi_nice.simplify()

(R**2*(R + epsilon) + 2*R*(R + epsilon)**2*log(R + epsilon) - R*(c1 - p1)**2 + 3*(R + epsilon)*(c1 - p1)**2)/(R*(R + epsilon)**2)

Putting everything together

Each component of a vector displacement field (in 3D here), we approximate with an RBF network

$$\Psi(\mathbf{p}) = w_0 + w_1 p_1 + w_2 p_2 + w_3 p_3 + \sum_{j=4}^{N_c} w_j \phi(r(\mathbf{p}, \mathbf{c}_j))$$

where $\mathbf{p} = [p_1, p_2, p_3]^T$, and $\phi(r)$ is a radial-basis function, in this case, a thin plate spline. We add the linear approximation $w_1 x_1 + w_2 x_2 + w_3 x_3$ because RBF interpolation is not exact for linear fields, which means the RBF approximation will likely have large errors for linear fields. Since rotation of a solid body is often used for mesh motion, and it is a linear field, it should be approximated accurately. Note that $w_1 p_1 + w_2 p_2 + w_3 p_3$ is a dot product, so the model can be easily programmed for n-dimensional vectors.

$$\partial_{p_i} \Psi(\mathbf{p}) = w_i + \sum_{j=4}^{N_c} w_j \partial_{p_i} \phi(r(\mathbf{p}, \mathbf{c_j}))$$

From the symbolic calculations above, we compute the gradient components

$$\partial_{p_i} \Psi(\mathbf{p}) = w_i + \sum_{j=4}^{N_c} w_j \frac{(c_i - p_i)(-r - 2(r + \epsilon)log(r+\epsilon))}{r + \epsilon} $$ 

for the each RBF network $\Psi$ used to approximate a component of the vector displacement field. This means $\Psi_i(\mathbf{x})$ approximates $\delta_i$ component of the displacement vector $\delta = [\delta_1, \delta_2, \delta_3]^T$. We need 3 RBF networks for 3 displacement-vector components.

The thin plate spline RBF approximation should be always constrained with "curvature" (Hessian), also computed symbolically: 

$$\partial^2_{p_i} \Psi(\mathbf{p}) = \sum_{j=4}^{N_c} w_j \partial^2_{p_i} \phi(r(\mathbf{p}, \mathbf{c}_j))$$ 

We re-use symbolic calculations above 

$$\partial^2_{p_i} \Psi(\mathbf{p}) = \sum_{j=4}^{N_c} w_j \frac{r^2 (r + \epsilon) + 2r(r+\epsilon)}{}$$ 









